Se cuentan con múltiples clústeres, cada uno de los cuales se dedica a temas específicos como salud, economía, turismo, delincuencia, etc.

Mi objetivo es identificar las palabras que conectan a un clúster A, enfocado por ejemplo en política, con un clúster B que trata por ejemplo sobre delincuencia. Como resultado, espero obtener los términos "ley", "refundar carabineros", "soborno" y "tráfico de influencias" como las palabras que los relacionan.

**Pregunta de investigación**: ¿Existe una relación léxica entre dos o más clústeres que hablan sobre distintos sectores?

**Hipótesis**: Se espera que exista una conexión léxica entre los clústeres que aparecen de manera frecuente en ambos.


#### Metodología

La hipótesis planteada sugiere que existe una relación léxica entre los clústeres de A y B, y se espera encontrar palabras en común que los relacionen. Para verificar esta hipótesis, se pueden realizar los siguientes pasos:

- Recopilar datos: Obtiene un conjunto de documentos.

- Procesamiento de lenguaje natural (NLP): Utiliza técnicas de procesamiento de lenguaje natural para analizar los textos y extraer palabras clave relevantes. Esto puede incluir la tokenización, eliminación de palabras vacías (stop words), lematización, entre otros. 

- Análisis de frecuencia de palabras: Calcular la frecuencia de ocurrencia de las palabras en los clústeres de política y delincuencia por separado. Identificar las palabras más frecuentes en cada clúster.

- Comparación de palabras clave: Identificar las palabras clave mencionadas en la pregunta de investigación ("ley", "refundar carabineros", "soborno" y "tráfico de influencias") y verificar su presencia y frecuencia en ambos clústeres.

- Análisis de co-ocurrencia: Analizar la co-ocurrencia de las palabras clave mencionadas en ambos clústeres para determinar si hay una relación léxica significativa: grafo no dirigido

#### Red de nodos

Enfoque utilizando un grafo no dirigido:

1. Construcción del grafo:

    - Cada clúster se representa como un conjunto de nodos.
    - Se crean conexiones entre los nodos basados en las palabras clave mencionadas ("ley", "refundar carabineros", "soborno" y "tráfico de influencias").
    
2. Ponderación de las conexiones:

    - Se asignan pesos a las conexiones basados en la frecuencia de ocurrencia de las palabras clave en los clústeres. Por ejemplo, si una palabra clave aparece con mayor frecuencia en ambos clústeres, se le asigna un peso más alto, lo que indica una relación más fuerte entre ellos.
    
3. Análisis del grafo:

    - Se utilizan algoritmos de grafos, como el algoritmo de detección de comunidades y/o el cálculo de centralidad, para identificar subconjuntos de nodos que tienen una conexión más fuerte. En este caso, se podrían buscar comunidades o agrupaciones que contengan las palabras clave mencionadas.

### 1. Importación de librerías y modelos 

In [2]:
import warnings
import pandas as pd
from tqdm import tqdm
import unidecode
import re
import string
from sentence_transformers import SentenceTransformer
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
import nltk
from nltk.corpus import stopwords
import spacy
import cohere
from bertopic.representation import Cohere

# Prevenir grandes mensajes de advertencia de BertModel
warnings.filterwarnings("ignore") 

# Importar paquetes
tqdm.pandas()
nltk.download('stopwords')
stopwords_es = stopwords.words('spanish')
nlp = spacy.load("es_core_news_sm")

# Leer archivo CSV
df = pd.read_csv("data/noticias-losrios.csv")
df.drop(df[df['text'].str.len() < 10].index, inplace=True)
df.drop(columns=["Unnamed: 0"], inplace=True)
df['content'] = df['title'] + "  " + df['text']
df['date'] = pd.to_datetime(df['date'])

[nltk_data] Downloading package stopwords to /home/ricki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Extract the unique months present in the date column
unique_months = df['date'].dt.month.unique()

# Filter the DataFrame based on the first month
first_month = unique_months[0]
first_month_data = df.loc[df['date'].dt.month == first_month]

# Filter the DataFrame based on the second month
second_month = unique_months[1]
second_month_data = df.loc[df['date'].dt.month == second_month]

# Concatenate the data for the first and second month
df = pd.concat([first_month_data, second_month_data])

In [4]:
print(len(df))
df

2187


,id_news,country,media_outlet,url,title,text,date,content
0,47314428.0,chile,suractual,https://www.suractual.cl/2022/01/24/ante-proye...,Ante proyecto Tierras Raras municipio y Unión ...,“Considerando la evolución que ha tenido el Pr...,2022-01-24,Ante proyecto Tierras Raras municipio y Unión ...
1,47719086.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/25/trac...,Tractocamión volcó esta mañana en puente Lloll...,El accidente se produjo pasadas las 06:00 AM y...,2022-01-25,Tractocamión volcó esta mañana en puente Lloll...
2,47719110.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/27/gobe...,Gobernadores Regionales de Los Ríos y Los Lago...,La instancia buscó generar un trabajo colabora...,2022-01-27,Gobernadores Regionales de Los Ríos y Los Lago...
3,47700339.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Alegan inocencia de acusadas por cruel asesina...,La defensa de las dos mujeres acusadas de enca...,2022-01-05,Alegan inocencia de acusadas por cruel asesina...
4,47700425.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Destacan ejecución del Programa Vínculos en Lo...,Para dar protección y acompañamiento a adultos...,2022-01-03,Destacan ejecución del Programa Vínculos en Lo...
...,...,...,...,...,...,...,...,...
2182,50286493.0,chile,diariocorral,https://www.diariocorral.cl/noticia/actualidad...,Polémica por proyecto fotovoltaico en Corral: ...,Por años las comunidades de Corral de los sect...,2022-02-26,Polémica por proyecto fotovoltaico en Corral: ...
2183,50289180.0,chile,diariocorral,https://www.diariocorral.cl/noticia/actualidad...,Seremi Pedro Lamas aclara dudas sobre la PGU e...,Un total de 41 mil 491 personas de la región d...,2022-02-01,Seremi Pedro Lamas aclara dudas sobre la PGU e...
2184,50300796.0,chile,lavozdepaillaco,https://www.lavozdepaillaco.cl/los-rios-colegi...,Los Ríos: Colegio Médico denuncia colapso asis...,El Colegio Médico de la Región de Los Ríos den...,2022-02-18,Los Ríos: Colegio Médico denuncia colapso asis...
2185,50300943.0,chile,lavozdepaillaco,https://www.lavozdepaillaco.cl/a-nivel-naciona...,A nivel nacional: Trabajadores recolectores de...,Después no digan que no la vieron venir. En me...,2022-02-18,A nivel nacional: Trabajadores recolectores de...


### 2. Preprocesamiento y extracción de ciudades

In [5]:
def clean_text(text):
    # Transformar texto a minúsculas
    text = text.lower()
    # Transformar caracteres acentuados a no acentuados
    text = unidecode.unidecode(str(text))
    # Borrar caracteres distintos a los alfabéticos
    text = re.sub(r'[^A-Za-z ]+', "", text) 
    # Borrar puntuaciones
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # Borrar palabras comprometidas con números
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
    # Borrar números
    text = re.sub(r'\d+', '', text)
    
    return text

def find_cities(text):
    cities = [
        "mariquina", "lanco", "mafil", "valdivia", "corral",
        "paillaco", "los lagos", "panguipulli", "la unión",
        "rio bueno", "lago ranco", "futrono"
    ]
    found = [city for city in cities if city in text] if text is not None else []
    
    return found

# Preprocesar texto
df['content'] = df.content.progress_apply(lambda doc: clean_text(str(doc)))

# Encontrar ciudades mencionadas en el contenido de noticias
df['cities'] = df.content.progress_apply(lambda doc: find_cities(doc))

# Obtener la lista de documentos
docs = df.content.tolist()

# Convertir la columna 'date' a formato de fecha
df['date'] = pd.to_datetime(df['date'])

# Imprimir el número de noticias procesadas
print("Number of news:", len(df))

# Obtener los tokens sin stopwords
no_stopwords = []
for doc in docs:
    tokens = []
    doc = nlp(doc)
    for token in doc:
        if token.text not in stopwords_es and len(token.text) > 2:
            tokens.append(token.text)
    no_stopwords.append(" ".join(tokens))

100%|████████████████████████████████████| 2187/2187 [00:00<00:00, 45266.96it/s]


Number of news: 2187


### 3. Extracción de Keywords con KeyBERT

In [6]:
# Extraer palabras clave
vectorizer = KeyphraseCountVectorizer(spacy_pipeline='es_core_news_sm')
kw_model = KeyBERT(model=SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2"))

df['keywords'] = df.content.progress_apply(lambda doc: kw_model.extract_keywords(doc,
                                                                                 vectorizer=vectorizer,
                                                                                 keyphrase_ngram_range=(1, 3),
                                                                                 use_mmr=True, 
                                                                                 diversity=0.7))

# Extraer las palabras clave como una lista
for index, row in df.iterrows():
    keywords = re.findall(r"'([^']*)'", str(row['keywords']))
    df.at[index, 'keywords'] = keywords

100%|███████████████████████████████████████| 2187/2187 [16:46<00:00,  2.17it/s]


### 4. Definición de la técnica de clustering BERTopic y representación de clusters con COHERE

In [7]:
# Definir los modelos de CountVectorizer, UMAP y HDBSCAN
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stopwords_es)
umap_model = UMAP(n_neighbors=10,
                  n_components=2,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=42)

hdbscan_model = HDBSCAN(min_cluster_size=10,
                        metric='euclidean',
                        cluster_selection_method='eom',
                        prediction_data=True,
                        min_samples=5)

# Create your representation model
with open('cohere_key.txt') as f:
    cohere_key = f.readlines()

co = cohere.Client(cohere_key[0])
PROMPT = """
Esta es una lista de textos donde cada colección de textos describe un tema relacionado con noticias en Chile. Después de cada colección de textos, se menciona el nombre del tema que representan como un título breve y altamente descriptivo.
--- 
Tema:
Textos de muestra de este tema:

- En las últimas décadas, la economía chilena ha experimentado un crecimiento constante, pero aún persisten desafíos en términos de desigualdad y acceso a oportunidades laborales.
- A pesar de los avances en materia de derechos humanos, Chile aún enfrenta críticas por casos de abusos policiales y violencia en las manifestaciones.
- La educación en Chile ha sido objeto de debate, con protestas estudiantiles exigiendo una educación gratuita y de calidad.

Palabras clave: economía desigualdad oportunidades derechos humanos abusos violencia educación protestas estudiantes calidad
Nombre del tema: Desafíos socioeconómicos en Chile
--- 
Tema:
Textos de muestra de este tema:

- La crisis sanitaria causada por la pandemia ha tenido un impacto significativo en la salud y la economía de Chile.
- El proceso de vacunación en Chile ha avanzado de manera exitosa, logrando altas tasas de inmunización en la población.
- Sin embargo, aún existen desafíos en cuanto a la equidad en la distribución de vacunas y la superación de la crisis sanitaria.

Palabras clave: crisis sanitaria pandemia salud economía vacunación inmunización población distribución equidad desafíos
Nombre del tema: Pandemia y vacunación en Chile
--- 
Tema:
Textos de muestra de este tema:
[DOCUMENTS]
Palabras clave: [KEYWORDS]
Nombre del tema:"""
representation_model = Cohere(co, prompt=PROMPT, delay_in_seconds=13)

# Definir el modelo de BERTopic
topic_model = BERTopic(n_gram_range=(1, 3),
                       top_n_words=15,
                       embedding_model=SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2"),
                       language="multilingual",
                       vectorizer_model=vectorizer_model,
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       representation_model=representation_model,
                       calculate_probabilities=False,
                       verbose=False)

# Obtener la lista de documentos sin stopwords
docs = df.content.tolist()

# Obtener los tokens sin stopwords
no_stopwords = []
for doc in docs:    
    doc = nlp(doc)
    tokens_per_doc = []
    for token in doc:
        if (token.text not in stopwords_es) and (len(token.text)>2):
            tokens_per_doc.append(token.text)
    elem = " ".join(tokens_per_doc)
    no_stopwords.append(elem)

In [8]:
topic_model.get_params()

{'calculate_probabilities': False,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': SentenceTransformer(
   (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
   (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
 ),
 'hdbscan_model': HDBSCAN(min_cluster_size=10, min_samples=5, prediction_data=True),
 'language': None,
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 3),
 'nr_topics': None,
 'representation_model': Cohere(client=<cohere.client.Client object at 0x7f4004f71870>,
        delay_in_seconds=13,
        prompt='\n'
               'Esta es una lista de textos donde cada colección de textos '
               'describe un tema relacionado con noticias en Chile. Después de '
               'cada colección de textos, se menciona el nombre del tema que '
          

In [9]:
topics, probs = topic_model.fit_transform(no_stopwords)
clusters = topic_model.get_topic_info()

# Asignamos a las noticias sus clusters correspondientes
T = topic_model.get_document_info(docs)
df = df.merge(T[['Document', 'Topic', 'Probability']], how='left', left_on='content', right_on='Document')
df.drop(columns=['content'], axis=1, inplace=True)

In [11]:
clusters

,Topic,Count,Name,Representation,Representative_Docs
0,-1,453,-1_Proyecto de vivienda en Ranco___,"[Proyecto de vivienda en Ranco, , , , , , , , , ]",[informe muestra percepcion economica mantiene...
1,0,155,0_Accidentes en ruta en Chile___,"[Accidentes en ruta en Chile, , , , , , , , , ]",[victima fatal dejo accidente vehicular sector...
2,1,73,1_Proyecto de vivienda en Chile___,"[Proyecto de vivienda en Chile, , , , , , , , , ]",[fiscalia rios rindio cuenta publica ceremonia...
3,2,72,2_Incendio en la Región Metropolitana___,"[Incendio en la Región Metropolitana, , , , , ...",[municipio habilita centro acopio ayuda bomber...
4,3,68,3_Crisis de niebla en Valparaíso___,"[Crisis de niebla en Valparaíso, , , , , , , ,...",[rios nuevas bibliotecas apuestan espacios lec...
...,...,...,...,...,...
68,67,10,67_Detención de mujer en Valparaíso___,"[Detención de mujer en Valparaíso, , , , , , ,...",[detienen valdivia mujer ordenes detencion pen...
69,68,10,68_Decreto condena proyectos inmobiliarios en ...,[Decreto condena proyectos inmobiliarios en el...,[cde prepara demanda proyectos inmobiliarios c...
70,69,10,69_Cerveza y fiesta en Chile___,"[Cerveza y fiesta en Chile, , , , , , , , , ]",[hoy domingo ultima noche fiesta cerveza paill...
71,70,10,70_COVID-19 en Chile___,"[COVID-19 en Chile, , , , , , , , , ]",[valdivia rio bueno anuncian testeos covid sab...


In [12]:
df

,id_news,country,media_outlet,url,title,text,date,cities,keywords,Document,Topic,Probability
0,47314428.0,chile,suractual,https://www.suractual.cl/2022/01/24/ante-proye...,Ante proyecto Tierras Raras municipio y Unión ...,“Considerando la evolución que ha tenido el Pr...,2022-01-24,[],"[proyecto tierras, democracia, consulta, horar...",ante proyecto tierras raras municipio y union ...,-1,0.000000
1,47719086.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/25/trac...,Tractocamión volcó esta mañana en puente Lloll...,El accidente se produjo pasadas las 06:00 AM y...,2022-01-25,[],"[puente, victimas, bomberos, semi remolque, al...",tractocamion volco esta manana en puente lloll...,0,1.000000
2,47719110.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/27/gobe...,Gobernadores Regionales de Los Ríos y Los Lago...,La instancia buscó generar un trabajo colabora...,2022-01-27,"[los lagos, rio bueno]","[gobernadores, ambas regiones, puyehue, turism...",gobernadores regionales de los rios y los lago...,15,0.839193
3,47700339.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Alegan inocencia de acusadas por cruel asesina...,La defensa de las dos mujeres acusadas de enca...,2022-01-05,[valdivia],"[dos acusados, helena bustos, dos mujeres, rio...",alegan inocencia de acusadas por cruel asesina...,46,1.000000
4,47700425.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Destacan ejecución del Programa Vínculos en Lo...,Para dar protección y acompañamiento a adultos...,2022-01-03,[valdivia],"[gobierno, rios, pandemia, envejecimiento, tal...",destacan ejecucion del programa vinculos en lo...,-1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2204,50286493.0,chile,diariocorral,https://www.diariocorral.cl/noticia/actualidad...,Polémica por proyecto fotovoltaico en Corral: ...,Por años las comunidades de Corral de los sect...,2022-02-26,"[valdivia, corral]","[luz, corral, vecinos, gobierno, tres chiflones]",polemica por proyecto fotovoltaico en corral n...,-1,0.000000
2205,50289180.0,chile,diariocorral,https://www.diariocorral.cl/noticia/actualidad...,Seremi Pedro Lamas aclara dudas sobre la PGU e...,Un total de 41 mil 491 personas de la región d...,2022-02-01,[corral],"[pension, mil chilenos, pobreza, rios, consultas]",seremi pedro lamas aclara dudas sobre la pgu e...,50,1.000000
2206,50300796.0,chile,lavozdepaillaco,https://www.lavozdepaillaco.cl/los-rios-colegi...,Los Ríos: Colegio Médico denuncia colapso asis...,El Colegio Médico de la Región de Los Ríos den...,2022-02-18,[],"[hospitalizaciones, rios, colapso, protegernos...",los rios colegio medico denuncia colapso asist...,31,0.851382
2207,50300943.0,chile,lavozdepaillaco,https://www.lavozdepaillaco.cl/a-nivel-naciona...,A nivel nacional: Trabajadores recolectores de...,Después no digan que no la vieron venir. En me...,2022-02-18,[],"[residuos, trabajadores recolectores, contagio...",a nivel nacional trabajadores recolectores de ...,11,0.824602


In [13]:
# Guardar en un CSV
df.to_csv("data/noticias-etiquetadas2.csv")
clusters.to_csv("data/clusters2.csv")